In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
#from sklearn.metrics import r2_score
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
print(device)
%matplotlib inline
np.random.seed(1)
plt.style.use("seaborn-talk") #"classic" "seaborn-talk" "seaborn"
path_to_h5 =  "/home/leonov/Baikal/Gr_big_data/mc_baikal_norm_cut-8_ordered_with_MCarlo.h5"

# можно отделить хвост и отдельно прогнать его по другой сетке
def make_trainset_noxyz(i, di = 1, tr_set_len = 128,Batch_size = 64):
    with h5py.File(path_to_h5, 'r') as hf:
            Data = hf['train/data/'][i*int(tr_set_len) : (i+di)*int(tr_set_len),:32]
            Data = torch.FloatTensor(Data.swapaxes(1, -1))
            Polar=np.expand_dims(hf["/train/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),0]*(np.pi)/180,
                                 axis=1)
            Polar_rec = np.expand_dims(hf["/train/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),3]*(np.pi)/180,
                                       axis=1)
            polar_error = torch.FloatTensor(Polar-Polar_rec)
            angles = torch.FloatTensor(np.concatenate((Polar,Polar_rec) ,axis=1))
            #Azimut=hf["/train/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),1]*(np.pi)/180
            #x=np.expand_dims(np.sin(Polar),axis=1)
            #y=np.expand_dims(np.cos(Polar),axis=1)
            #target=torch.FloatTensor(np.concatenate((x,y) ,axis=1))
            # надо,т.к. второй индекс должен быть количеством   последовательностей
            trainDataset = torch.utils.data.TensorDataset(Data, polar_error, angles) #target
            trainLoader = torch.utils.data.DataLoader(dataset = trainDataset, batch_size=Batch_size, drop_last = True) #,sampler = sampler    
    return  trainLoader
def make_valset_noxyz(i=0, di = 1, tr_set_len = 1000, Batch_size = 64):
    with h5py.File(path_to_h5, 'r') as hf:
            Data = hf['val/data/'][i*int(tr_set_len) : (i+di)*int(tr_set_len),:32]
            Data = torch.FloatTensor(Data.swapaxes(1, -1))
            Polar=np.expand_dims(hf["/val/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),0]*(np.pi)/180,
                                 axis=1)
            Polar_rec = np.expand_dims(hf["/val/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),3]*(np.pi)/180,
                                       axis=1)
            polar_error = torch.FloatTensor(Polar-Polar_rec)
            #x=np.expand_dims(np.sin(Polar),axis=1)
            #y=np.expand_dims(np.cos(Polar),axis=1)
            #target=torch.FloatTensor(np.concatenate((x,y) ,axis=1))
            testDataset = torch.utils.data.TensorDataset(Data, polar_error, angles)
            testLoader = torch.utils.data.DataLoader(dataset = testDataset, batch_size=Batch_size, drop_last = True)
    return testLoader
# вектора в углы
def v_to_angles( Predicted, Real, Angles, #predicted - предсказанное отклонение реконструкции от угла /Real =Pol-Pol_reconst
                p_hist, p_error_hist, p_error_hist_abs, # Angles: 0- real, 1- reconstruct
                p_error_angle_cut_hist, I_want_scatter_plot = False,
                min_angle = 10., max_angle = 60.): 
    final_error = Real - Predicted
    final_rec_angle = Angles[:,1] + Predicted
    real_angle = Angles[:,0]
    if I_want_scatter_plot:
        plt.scatter(real_angle.cpu().detach().numpy(),final_rec_angle.cpu().detach().numpy(), s=1 ,color = "blue",alpha =0.3 )
        plt.xlabel("Real polar angle", fontsize = 18);plt.ylabel("Updated reconstructed polar angle", fontsize = 18)
        plt.title("Scatter plot for polar angle", fontsize = 24)
    for error, pol_real,pol_final in zip(final_error,real_angle,final_rec_angle):
        p_error_hist[round(error.item(),1)] += 1
        p_error_hist_abs[round(abs(error.item()),1)] += 1
        # for certain angles !!!!!!!!!!!!!!!!!!!!!!!
        if (pol_real.item() >= min_angle) and (pol_real.item() <= max_angle): 
            p_error_angle_cut_hist[round(abs(error.item()),1)] += 1
        p_hist[pol_final.short().item()] += 1
    

def loss_plot(list_test, list_train , path , save = True):
    plt.figure(figsize=(9,6))
    plt.plot(np.arange(len(list_test)), list_test, label='val', linewidth=2)
    plt.plot(np.arange(len(list_train)), list_train, label='train', linewidth=2)
    plt.title('Loss_plot')
    plt.xlabel('iterations')
    plt.ylabel('Loss')
    plt.legend()
    if save == True:
        plt.savefig(path)
    plt.show()
def res_plot(train_dict,val_dict, path = None,save = True, res_or_polar = "Resolution " ,size= 13):
    plt.figure(figsize=(13,5))
    colours=["red","red"]
    names = ["train" , "val"]
    for i, d in enumerate([train_dict,val_dict]):
        plt.subplot(1,2,i+1)
        s = sum(d.values())
        prep_inter_s ,inter_s = 0, 0
        res_50, res_68 =0,0
        for key in list(d.keys()):
            if inter_s/s >= 0.68:
                alpha = (inter_s-0.68*s)/(inter_s-prep_inter_s) # alpha*inter_s+(1-alpha)*prep_inter_s == 0.68*s
                res_68 =round(key + 0.1*alpha,2)  #alpha*(key-0.1)+(1-alpha)*(key)
                break
            if inter_s/s >= 0.5 and res_50 == 0:
                alpha = (inter_s-0.5*s)/(inter_s-prep_inter_s)
                res_50 =round(key + 0.1*alpha,2)
            prep_inter_s = inter_s 
            inter_s  += d[key]  
        a=plt.step(list(d.keys())[:300], list(d.values())[:300], color=colours[i],alpha = 0.6)
        plt.bar(res_50, max(d.values()), width=0.5,label=names[i] + "50%" + res_or_polar +"= "+str(res_50),
                color="yellow" , alpha =0.7 )
        plt.bar(res_68, max(d.values()), width=0.5,label=names[i] + "68%" + res_or_polar +" = "+str(res_68) ,
                color="orange",alpha = 0.7)
        plt.legend(fontsize= 12); #plt.suptitle(res_or_polar,fontsize =size+3)
        plt.xlabel(res_or_polar + "in_grad",fontsize= 16); plt.title(res_or_polar + names[i],fontsize= 22)
    if save == True:
        plt.savefig(path) 
    plt.show()

def angle_hist(hist_polar,path, save =True,size= 13,name = "train"): #hist_azimut
    with h5py.File(path_to_h5, 'r') as hf:
        plt.figure(figsize= (13,6))
        sum_value = sum(hist_polar.values())
        Polar=hf["/"+name+"/ev_chars"][ : ,0] 
        #Azimut=hf["/"+name+"/ev_chars"][ : ,1]
        #plt.subplot(2,1,1)
        plt.hist(Polar,bins=180,label="Real polar angle in " + name +" data",density=True,histtype="step",color="blue")
        plt.bar(list(hist_polar.keys())[:100], np.array(list(hist_polar.values())[:100])/sum_value, 
                color="red",label= "Predicted polar angle in " + name +" data")
        plt.legend(fontsize = 18)
        plt.xlabel("Polar angle",fontsize= 25); plt.title(name+" polar angle",fontsize= 30)
        if save == True:
            plt.savefig(path) 
        plt.show()
